In [ ]:
from flask import Flask
from flask import request
import nltk.stem.porter as stemmer
import json
import requests

app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

@app.route("/analyze", methods=["POST"])
def analyze():
    j = request.json
    # query format - string
    query = j['data']
    
    #language detection
    response_languagedetector = requests.post('http://127.0.0.1:13534/analysis/detector', json={'data' : query})
    language = json.loads(response_languagedetector.text)['language']
    
    #tokenizing
    response_tokenized = requests.post('http://127.0.0.1:13549/analysis/tokenize', json={'data' : query})
    parsed_tokenized = json.loads(response_tokenized.text)
    
    #stemming
    response_stemmed = requests.post('http://127.0.0.1:13535/analysis/stemmer', json={'data' : parsed_tokenized['processed_data']})
    parsed_stemmed = json.loads(response_stemmed.text)
    
    #delete stopwords
    response_stopwords = requests.post('http://127.0.0.1:13536/analysis/stopwords', json={'data' : parsed_stemmed['processed_data']})
    parsed_stopwords = json.loads(response_stopwords.text)
    
    #add synonymus
    response_synonymus = requests.post('http://127.0.0.1:13539/analysis/synonymus', json={'data' : parsed_stopwords['processed_data']})
    parsed_synonymus = json.loads(response_synonymus.text)
    
    #fix misprints
    response_fixed = requests.post('http://127.0.0.1:13540/analysis/misprints', json={'data' : parsed_stopwords['processed_data']})
    parsed_fixed_text = json.loads(response_fixed.text)
    return json.dumps({"status":"ok", "words": parsed_fixed_text['processed_data'], "language" : language})
    

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=13533)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:13533/ (Press CTRL+C to quit)
